L'idea successiva è di fare 4 finetuning diversi su 4 segmenter partendo da modelli già trainati, uno per ogni tipo di scan (flair, t1, t2, t1ce)

In questo notebook mettiamo le basi, cioè creiamo i 4 dataset che ci servono

Nel prossimo notebook proveremo ad utilizzare un segmenter già pre-trainato e testarlo sulle immagini "flair" del BraTS dataset (slice)

In [2]:
import os
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image

SEED = 1234
tf.random.set_seed(SEED)

cwd = os.getcwd() #get current working directory
physical_devices = tf.config.list_physical_devices('GPU') 
#physical_devices = tf.config.list_physical_devices('TPU') 

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!pip install git+https://github.com/rstrudel/segmenter.git
#!pip install .

In [ ]:
#!pip install --ignore-installed PyYAML 

# RUNNING


Col codice che segue vorremmo prendere i file di tipo tfrecords e decodificarli per ottenerne le immagini. Ogni paziente ha 4 immagini (più la segmentazione) ad un canale (grayscale).
Le colormaps si possono trovare al sito https://scipy-cookbook.readthedocs.io/items/Matplotlib_Show_colormaps.html

Fatto questo, le salviamo nelle diverse sottocartelle di Extract come immagini a 3 canali.

In [3]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import numpy as np
import cv2

#dataset_folder = os.path.join(cwd, "drive/My Drive/Unzipped/");
#dataset_name = os.path.join(dataset_folder, "brats2019_slice_training_tfrecords/brats2019_slice_training.tfrecords")
dataset_name = os.path.join('/content/drive/My Drive/Unzipped/', "brats2019_slice_training_tfrecords/brats2019_slice_training.tfrecords")
#dataset_name = os.path.join('/content/drive/My Drive/Unzipped/', "brats2019_slice_validation_tfrecords/brats2019_slice_validation.tfrecords")
raw_dataset = tf.data.TFRecordDataset(dataset_name)

count = 0
for raw_record in raw_dataset.take(2000): #così prende solo le prime 10
#for raw_record in raw_dataset: #così prende invece tutto il dataset
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())
    #print(example)
    #print(repr(raw_record))


    parsed = tf.train.Example.FromString(raw_record.numpy())
    feature = parsed.features.feature

    raw_img_flair_mri = feature['flair_mri'].bytes_list.value[0]
    #raw_img_t1_mri = feature['t1_mri'].bytes_list.value[0]
    #raw_img_t1ce_mri = feature['t1ce_mri'].bytes_list.value[0]
    #raw_img_t2_mri = feature['t2_mri'].bytes_list.value[0]
    raw_img_seg = feature['seg'].bytes_list.value[0]

    img_flair_mri = tf.image.decode_png(raw_img_flair_mri)
    #img_t1_mri = tf.image.decode_png(raw_img_t1_mri)
    #img_t1ce_mri = tf.image.decode_png(raw_img_t1ce_mri)
    #img_t2_mri = tf.image.decode_png(raw_img_t2_mri)
    img_seg = tf.image.decode_png(raw_img_seg)

    #f, splt = plt.subplots(1,5, figsize = (25.0, 25.0)) 

    
    #splt[0].imshow(tf.squeeze(img_flair_mri), cmap='jet', vmin=0, vmax=255)
    #splt[1].imshow(tf.squeeze(img_t1_mri), cmap='jet')
    #splt[2].imshow(tf.squeeze(img_t2_mri), cmap='jet')
    #splt[3].imshow(tf.squeeze(img_t1ce_mri), cmap='jet')
    #splt[4].imshow(tf.squeeze(img_seg), cmap='jet')

    color_map = plt.get_cmap('jet')

    colored_image = color_map(np.array(tf.squeeze(img_flair_mri)))[:,:,:3]
    colored_image = np.uint8(colored_image * 255)
    #Image.fromarray(colored_image).save('/content/drive/My Drive/Extracted/flair/images/validation/flair_{}.png'.format(count))
    #Image.fromarray(colored_image).save('/content/drive/My Drive/Extracted_reduced/flair/images/validation/flair_{}.png'.format(count))
    Image.fromarray(colored_image).save('/content/drive/My Drive/Extracted_reduced/flair/images/training/flair_{}.png'.format(count))

    #colored_image = color_map(np.array(tf.squeeze(img_t1_mri)))[:,:,:3]
    #colored_image = np.uint8(colored_image * 255)
    #Image.fromarray(colored_image).save('/content/drive/My Drive/Extracted/t1/images/validation/t1_{}.png'.format(count))

    #colored_image = color_map(np.array(tf.squeeze(img_t2_mri)))[:,:,:3]
    #colored_image = np.uint8(colored_image * 255)
    #Image.fromarray(colored_image).save('/content/drive/My Drive/Extracted/t2/images/validation/t2_{}.png'.format(count))
   
    #colored_image = color_map(np.array(tf.squeeze(img_t1ce_mri)))[:,:,:3]
    #colored_image = np.uint8(colored_image * 255)
    #Image.fromarray(colored_image).save('/content/drive/My Drive/Extracted/t1ce/images/validation/t1ce_{}.png'.format(count))



    #with np.printoptions(threshold=np.inf): #se devo stampare tutto il contenuto dell'immagine, scommento questo e indento la parte rimanente


    #colored_image = np.uint8(np.squeeze(img_seg) * 255.0/4)
    #Image.fromarray(colored_image).save('/content/drive/My Drive/Extracted/flair/annotations_for_humans/validation/flair_{}.png'.format(count))
    #Image.fromarray(colored_image).save('/content/drive/My Drive/Extracted/t1/annotations_for_humans/validation/t1_{}.png'.format(count))
    #Image.fromarray(colored_image).save('/content/drive/My Drive/Extracted/t2/annotations_for_humans/validation/t2_{}.png'.format(count))
    #Image.fromarray(colored_image).save('/content/drive/My Drive/Extracted/t1ce/annotations_for_humans/validation/t1ce_{}.png'.format(count))

    
    colored_image = np.uint8(np.squeeze(img_seg))
    #print(np.unique(colored_image))
    #Image.fromarray(colored_image).save('/content/drive/My Drive/Extracted_reduced/flair/annotations/validation/flair_{}.png'.format(count))
    Image.fromarray(colored_image).save('/content/drive/My Drive/Extracted_reduced/flair/annotations/training/flair_{}.png'.format(count))
    #Image.fromarray(colored_image).save('/content/drive/My Drive/Extracted/t1/annotations/validation/t1_{}.png'.format(count))
    #Image.fromarray(colored_image).save('/content/drive/My Drive/Extracted/t2/annotations/validation/t2_{}.png'.format(count))
    #Image.fromarray(colored_image).save('/content/drive/My Drive/Extracted/t1ce/annotations/validation/t1ce_{}.png'.format(count))
    #image_seg = cv2.imread('/content/drive/My Drive/Extracted/flair/annotations/training/flair_{}.png'.format(count))
    #print(np.unique(image_seg))
    
    print("Image {} processed".format(count))
    count+=1


Image 0 processed
Image 1 processed
Image 2 processed
Image 3 processed
Image 4 processed
Image 5 processed
Image 6 processed
Image 7 processed
Image 8 processed
Image 9 processed
Image 10 processed
Image 11 processed
Image 12 processed
Image 13 processed
Image 14 processed
Image 15 processed
Image 16 processed
Image 17 processed
Image 18 processed
Image 19 processed
Image 20 processed
Image 21 processed
Image 22 processed
Image 23 processed
Image 24 processed
Image 25 processed
Image 26 processed
Image 27 processed
Image 28 processed
Image 29 processed
Image 30 processed
Image 31 processed
Image 32 processed
Image 33 processed
Image 34 processed
Image 35 processed
Image 36 processed
Image 37 processed
Image 38 processed
Image 39 processed
Image 40 processed
Image 41 processed
Image 42 processed
Image 43 processed
Image 44 processed
Image 45 processed
Image 46 processed
Image 47 processed
Image 48 processed
Image 49 processed
Image 50 processed
Image 51 processed
Image 52 processed
Ima

In [ ]:
%cd '/content/drive/My Drive/Extracted/Flair/annotations/training'
%ls 

[Errno 2] No such file or directory: '/content/drive/My Drive/Extracted/Flair/annotations/training'
/content
drive/  sample_data/
